# Using `fmriprep` on the SLURM cluster



--------------------


#### HISTORY

* 7/9/2021 - dcosme revised for bbprime
* 5/22/20 - mbod updated for revised MURI fmriprep pipeline
* 1/22/20 - matt, jeesung and nicole initial test setup for using SLURM

In [2]:
import os

## 1. Create job files

* Each process that you send to SLURM needs a job file, which is a text file with the commands to be executed by SLURM

* For the bbprime project these job files should be placed in:

```
/data00/projects/bbprime/scripts/FMRIPREP/jobs
```

* A job file has:

    1. some SLURM directives in lines beginning `#SBATCH`
    2. one or more commands to be run (prefixed with `srun`)
    
```
#!/bin/bash
#SBATCH --job-name=fmriprep_{ID}.job
#SBATCH --output=out/fmriprep_{ID}.out
#SBATCH --error=out/fmriprep_{ID}.err
#SBATCH --time=2-00:00

srun singularity run --cleanenv \
    -B /data00/tools/freesurfer/license.txt:/opt/freesurfer/license.txt  \
    -B /data00/projects/bbprime/data/bids_data:/data \
    -B /data00/projects/bbprime/data/bids_data/derivatives:/out  \
    -B /data00/projects/bbprime/data/bids_data/derivatives/working:/work \
    /data00/tools/singularity_images/fmriprep_v20.0.6 /data /out -w /work \
    --ignore slicetiming \
    --nthread 8 \
    participant --participant_label {ID} \
    --skip-bids-validation
```

### Define variables

In [67]:
project = 'bbprime'
project_dir = os.path.join('/data00/projects/' + project)
slurm_dir = os.path.join(project_dir + '/scripts/FMRIPREP/jobs')

### Make the directory if it doesn't exist

In [68]:
!mkdir $slurm_dir

mkdir: cannot create directory ‘/data00/projects/bbprime/scripts/FMRIPREP/jobs’: File exists


### Create job template

In [8]:
job_file_template = r'''#!/bin/bash
#SBATCH --job-name=fmriprep_{ID}.job
#SBATCH --output=out/fmriprep_{ID}.out
#SBATCH --error=out/fmriprep_{ID}.err
#SBATCH --time=2-00:00

srun singularity run --cleanenv \
    -B /data00/tools/freesurfer/license.txt:/opt/freesurfer/license.txt  \
    -B /data00/projects/bbprime/data/bids_data:/data \
    -B /data00/projects/bbprime/data/bids_data/derivatives:/out  \
    -B /data00/projects/bbprime/data/bids_data/derivatives/working:/work \
    /data00/tools/singularity_images/fmriprep_v20.0.6 /data /out -w /work \
    --ignore slicetiming \
    --nthread 8 \
    participant --participant_label {ID} \
    --skip-bids-validation
'''


In [9]:
print(job_file_template.format(ID='sub-BPP00'))

#!/bin/bash
#SBATCH --job-name=fmriprep_sub-BPP00.job
#SBATCH --output=out/fmriprep_sub-BPP00.out
#SBATCH --error=out/fmriprep_sub-BPP00.err
#SBATCH --time=2-00:00

srun singularity run --cleanenv \
    -B /data00/tools/freesurfer/license.txt:/opt/freesurfer/license.txt  \
    -B /data00/projects/bbprime/data/bids_data:/data \
    -B /data00/projects/bbprime/data/bids_data/derivatives:/out  \
    -B /data00/projects/bbprime/data/bids_data/derivatives/working:/work \
    /data00/tools/singularity_images/fmriprep_v20.0.6 /data /out -w /work \
    --ignore slicetiming \
    --nthread 8 \
    participant --participant_label sub-BPP00 \
    --skip-bids-validation



### Identify participants who lack job files
Of the participants listed in the `{project_dir}/data/bids_data`, identify those who lack a job file and create it

In [63]:
subs = [s for s in os.listdir(os.path.join(project_dir, 'data/bids_data'))
                       if s.startswith('sub-BP')]

existing_job_files = [s for s in os.listdir(slurm_dir)
                       if s.endswith('.job') and s.count('sub-BP')==1]

subs_to_process = [sub for sub in subs if 'fmriprep_' + sub + '.job' not in existing_job_files]

### Create a job file for each participant in `subs_to_process`

In [15]:
for s in subs_to_process:
    file_path = os.path.join(slurm_dir, 'fmriprep_{}.job').format(s)
    print('Creating', file_path)

    with open(file_path.format(s), 'w') as job:
        job.write(job_file_template.format(ID=s))


Creating /data00/projects/bbprime/scripts/FMRIPREP/jobs/fmriprep_sub-BPP00.job
Creating /data00/projects/bbprime/scripts/FMRIPREP/jobs/fmriprep_sub-BPP01.job
Creating /data00/projects/bbprime/scripts/FMRIPREP/jobs/fmriprep_sub-BPP05.job
Creating /data00/projects/bbprime/scripts/FMRIPREP/jobs/fmriprep_sub-BPP06.job
Creating /data00/projects/bbprime/scripts/FMRIPREP/jobs/fmriprep_sub-BPP07.job
Creating /data00/projects/bbprime/scripts/FMRIPREP/jobs/fmriprep_sub-BPP11.job
Creating /data00/projects/bbprime/scripts/FMRIPREP/jobs/fmriprep_sub-BPP13.job
Creating /data00/projects/bbprime/scripts/FMRIPREP/jobs/fmriprep_sub-BPP15.job
Creating /data00/projects/bbprime/scripts/FMRIPREP/jobs/fmriprep_sub-BPP19.job
Creating /data00/projects/bbprime/scripts/FMRIPREP/jobs/fmriprep_sub-BPP20.job
Creating /data00/projects/bbprime/scripts/FMRIPREP/jobs/fmriprep_sub-BPP21.job
Creating /data00/projects/bbprime/scripts/FMRIPREP/jobs/fmriprep_sub-BPP22.job
Creating /data00/projects/bbprime/scripts/FMRIPREP/j

## 2. Submit jobs to SLURM cluster

### How to log onto the SLURM master


* master name: `cls000.asc.upenn.edu`


```
ssh <JANUS_UN>@asc.upenn.edu@cls000
```


* This will give you a terminal on the SLURM master node where you can look at the process queue and schedule jobs.

### How to schedule a job to SLURM

* Use the `sbatch` command

    ```
    sbatch -D /data00/slurm_jobs/slurm_bbprime -c 8 name_of_job_file
    
    ```

    * `-D` is the working directory
    * `-c` the number of processors to use
    * then the name of the job file

### Create log file directory

If this doesn't exist, your job will fail without explanation

In [16]:
!mkdir $slurm_dir/out

### Run all jobs
To submit all the jobs, copy and past the following lines of code into the terminal after logging into the slurm cluster

In [17]:
print("Alternatively, submit the whole batch by copying and pasting the following lines in terminal:")
print("")

print(f"cd {slurm_dir}")
for s in subs_to_process:
    print(f"sbatch -D {slurm_dir} -c 8 fmriprep_{s}.job")
print(" ")

Alternatively, submit the whole batch by copying and pasting the following lines in terminal:

cd /data00/projects/bbprime/scripts/FMRIPREP/jobs
sbatch -D /data00/projects/bbprime/scripts/FMRIPREP/jobs -c 8 fmriprep_sub-BPP00.job
sbatch -D /data00/projects/bbprime/scripts/FMRIPREP/jobs -c 8 fmriprep_sub-BPP01.job
sbatch -D /data00/projects/bbprime/scripts/FMRIPREP/jobs -c 8 fmriprep_sub-BPP05.job
sbatch -D /data00/projects/bbprime/scripts/FMRIPREP/jobs -c 8 fmriprep_sub-BPP06.job
sbatch -D /data00/projects/bbprime/scripts/FMRIPREP/jobs -c 8 fmriprep_sub-BPP07.job
sbatch -D /data00/projects/bbprime/scripts/FMRIPREP/jobs -c 8 fmriprep_sub-BPP11.job
sbatch -D /data00/projects/bbprime/scripts/FMRIPREP/jobs -c 8 fmriprep_sub-BPP13.job
sbatch -D /data00/projects/bbprime/scripts/FMRIPREP/jobs -c 8 fmriprep_sub-BPP15.job
sbatch -D /data00/projects/bbprime/scripts/FMRIPREP/jobs -c 8 fmriprep_sub-BPP19.job
sbatch -D /data00/projects/bbprime/scripts/FMRIPREP/jobs -c 8 fmriprep_sub-BPP20.job
sbatc